In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Stats import WeightQuantization,IntroduceFaultsInWeights,GenerateFaultsList
from Nets import GetNeuralNetworkModel
from Training import GetDatasets
from Simulation import get_all_outputs

In [ ]:
Estudio d elos pesos

In [ ]:
ok probar las tecnica que tenemos con errores en los pesos
Tamaño de los pesos por capas y por redes
llevar los Nan a 0 a ver que ocurre
crear mapa de fallo de pesos
analizar si comienzo dede la posición 0 en adelante hasta el final d ela capa o continuo

In [2]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Stats import WeightQuantization,IntroduceFaultsInWeights,GenerateFaultsList
#from Stats_original import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList, CheckAccuracyAndLoss
from Nets import GetNeuralNetworkModel
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds

### Definiciones

In [3]:
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4
# Tamaño del buffer de pesos == al tamaño de la capa con mayor numero de pesos (885120 pesos de 16 bits cada uno)
wbuffer_size = 885120*word_size
# Tamaño del buffer de activaciones == al tamaño de la capa con mayor numero de activaciones (290400 pesos de 16 bits cada uno)
abuffer_size = 290400*word_size
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

### Generacion de una muestra de fallos

In [4]:
# Decidir si cargar errores de un archivo locs y error_mask o generarlos aleatoriamente
Cargar_errores = False

if Cargar_errores:
    locs  = load_obj('locs')
    masks = load_obj('error_mask')
else:
    numero_bits_con_fallo = 1000
    bits_con_fallo = np.random.randint(0,2,numero_bits_con_fallo)
    #crear una mascara (m) del buffer de pesos donde x: bit sin fallo, 0: bit con fallo en 0, 1: bit con fallo en 1.
    #si quieres introducir fallos en activaciones en lugar de los pesos, simplemente cambia wbuffer_size por abuffer_size.
    mbuffer = np.array(['x']*(wbuffer_size-numero_bits_con_fallo))
    mbuffer = np.concatenate([mbuffer,bits_con_fallo])
    #distribuir los errores aleatoriamente en la mascara del buffer
    np.random.shuffle(mbuffer)
    #organizar la mascara del buffer por direcciones
    mbuffer = np.reshape(mbuffer,(-1,word_size))
    mbuffer = ["".join(i) for i in mbuffer]
    #filtrar dejando solo las direcciones con error
    locs  = [x for x,y in enumerate(mbuffer) if y.count('x') < 16]
    masks = [y for x,y in enumerate(mbuffer) if y.count('x') < 16] 
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:5])
print('mascara de fallos:',masks[0:5])

mostrando las 5 primeras direcciones con fallos
direcciones: [483, 1393, 1629, 2887, 2888]
mascara de fallos: ['1xxxxxxxxxxxxxxx', 'xxxxxxxxxxx1xxxx', '0xxxxxxxxxxxxxxx', 'xxxx0xxxxxxxxxxx', 'xxxxxxxxxxxxxxx0']


### Cargar el dataset

In [4]:
trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)

C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


### Crear la red sin fallos

In [5]:
activation_aging = False

#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net1 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net1.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net1, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net1.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
pesos1 = Net1.get_weights()
Net1.evaluate(test_dataset)

47/47 [==============================] - 17s 363ms/step - loss: 0.3125 - accuracy: 0.8907


[0.3124775290489197, 0.890666663646698]

In [5]:
#index=3
#weights = Net1.layers[index].get_weights()
#capa=Net1.layers[index].__class__.__name__
#print(capa)
#print(weights)

In [18]:
Net1.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
lambda (Lambda)              (None, 227, 227, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 227, 227, 3)       0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
lambda_2 (Lambda)            (None, 55, 55, 96)        0         
_________________________________________________________________
re_lu (ReLU)                 (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384   

In [12]:
Net1.layers[0].__class__.__name__

'InputLayer'

In [ ]:
for index in range(0,len(outputs2)):
    print('Capa',index,Net2.layers[index].__class__.__name__)
    #a=(np.count_nonzero(outputs2[index]))
    #b=(outputs2[index].size)
    a=outputs1[index]== outputs2[index]
    size_output=a.size
    print('Cantidad de elementos de la capa:',size_output)
    output_true=np.sum(a)
    #print('Cantidad de elementos iguales entre modelo con fallos y sin fallos:', output_true)
    amount_dif=size_output-output_true
    print('Cantidad de elementos diferentes entre modelo con fallos y sin fallos', amount_dif)
    ratio=(output_true*100)/size_output
    ratio_dif=(amount_dif*100)/size_output
    #print('Accurancy:', ratio,'%')
    print('Ratio:', ratio_dif,'%')

## variante redimensionando

In [7]:
activation_aging = False

#cambiare la direccion del primer error y la posicion del fallo,
#seteandolo en una direccion baja (que afecte a la capa de normalizacion) y así poder observar la caida
#locs[0] = 0
#masks[0] = 'xx1xxxxxxxxxxxxx'
error_mask_address = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/error_mask_054')
locs_address = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/locs_054')


#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
#Introducir fallas en cada capa de pesos
weights = Net2.get_weights()
print(len(weights))
#weights_0 = Net2.layers[0].get_weights()
##print(weights_0)
#for index,layer in enumerate(Net2.layers):
#    print(Net2.layers[index].__class__.__name__)
#    
#    if(len(Net2.layers[index].weights) > 0):
#        if layer.__class__.__name__ in ['Conv2D','Conv1D','DepthwiseConv2D']:
#        #if Net2.layers[index].__class__.__name__=='Conv2D':
#            weights, biases = Net2.layers[index].get_weights()
#            weights_conv = np.reshape(weights, (-1))
#            print('shape weights_conv',(weights_conv.shape))
#            join_weights_conv_bias=np.concatenate((weights_conv, biases))
#            print('join perfecto', join_weights_conv_bias.shape)
#            
#        if  Net2.layers[index].__class__.__name__=='BatchNormalization':
#            gamma,beta,moving_mean,moving_std = Net2.layers[index].get_weights()
#            gamma = np.reshape(gamma, (-1))
#            beta = np.reshape(beta, (-1))
#            moving_mean = np.reshape(moving_mean, (-1))
#            moving_std = np.reshape(moving_std, (-1))
#            weights_1=np.concatenate((gamma,beta,moving_mean,moving_std))
#            print('shape de weights_1:', weights_1.shape)  
#         
#        if  Net2.layers[index].__class__.__name__=='Dense':  
#            weights_dense, biases_dense = Net2.layers[index].get_weights()
#            weights_dense = np.reshape(weights_dense, (-1))
#            biases_dense = np.reshape(biases_dense, (-1))
#            weights_2=np.concatenate((weights_dense, biases_dense))
#            print('weights_2', weights_2.shape)
### El tema está que a la hora d einyectar  los fallos como está diseñado se hace al modleo y de esta forma sería por capas
###entonces en este caso cuando se procese una capa deberiamos de mandar a inyectar los errores y luego agregarlo al modelo 
### ahi no entiendo como manejarlo
#positionList,faultList,NumberOfFaults = GenerateFaultsList(shape=itm.shape,locs=faulty_addresses,error_mask=masked_faults)
#
#    if NumberOfFaults > 0:
#        weights[index] = IntroduceFaultsInWeights(itm,positionList,faultList,wint_size,wint_size)
#qNet.set_weights(weights)
##loss = tf.keras.losses.CategoricalCrossentropy()
##optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
##Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
##loss,acc  = Net2.evaluate(test_dataset)
#                    
#            
        

36


## Algoritmo

In [ ]:
Algoritmo(crear una función que tome el modelo)
le pido los pesos al modelo
recorro las el modelo por capas
si la capa tiene pesos y es convolución:
    calculo la candidad de elementos que tiene la variable weights.size
    lo guardo en un variable
    la vectorizo a 1 dimensión y a concateno con el bias 
    le paso ese itm a la función IntroduceFaultsInWeights
    luego la separos y las coloco son el shape original 
    y a model.layers[index] le establezco los pesos ya con fallos
si la capa tiene pesos y es Normalizatión:
    tiene los parametros gamma,beta,moving_mean,moving_std
    hago lo mismo por cada elemento
    (seria mejor hacer dos funciones nuevas para que hagan lo de vectorizar y luego separar de nuevo)
    model.layers[index] le establezco los pesos ya con fallos
Si es un capa Dense:
    tiene weights y bias
    hago los mismo 
    model.layers[index] le establezco los pesos ya con fallos
la función devuelve el modelo para qu econtinue el algoritmo   

para comprobar que funciona imprimo las variables positionList,faultLis




## Función

In [47]:
activation_aging = False

error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/error_mask_054')
locs = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/locs_054')
wfrac_size = 11
wint_size  = 4
#locs = [0]
#error_mask = ['xx1xxxxxxxxxxxxx']

#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
for index,layer in enumerate(Net2.layers):
   
    
    weights_tmp=[]
    #print(Net2.layers[index].__class__.__name__)
        
    if(len(Net2.layers[index].weights) > 0):
        print(Net2.layers[index].__class__.__name__)
        if layer.__class__.__name__ in ['Conv2D','Conv1D','DepthwiseConv2D']:
            print('tamaño d ela capa' ,layer.output_shape)
            
            weights, biases = Net2.layers[index].get_weights()
                
            w_shape= weights.shape
            b_shape= biases.shape
            print('shape wights_conv_antes',w_shape)
            print('shape bias_antes_conv',b_shape)
            wsize=weights.size
            weights_conv = np.reshape(weights, (-1))
            weig_conv_bias=np.concatenate((weights_conv, biases))
            itm= weig_conv_bias
            positionList,faultList,NumberOfFaults = GenerateFaultsList(shape=itm.shape,locs=locs,error_mask=error_mask)
            itm_fail= IntroduceFaultsInWeights(itm,positionList,faultList,wfrac_size,wint_size) 
            
            print('numero de fallos insertados', NumberOfFaults)
            #print('lista de posiciones', positionList )
            #print('shape de layer en 0',layer.output_shape[0])
            layer_size=layer.output_shape[1]*layer.output_shape[2]*layer.output_shape[3]
            print('tamaño de del array weights',wsize)
            print('cantidad de pocisiones afectada capa',np.count_nonzero(positionList <= wsize))
            print('tamaño de la lista d eposiciones', len(positionList))
            #d = itm_fail[:wsize]
            w=np.reshape(itm_fail[:wsize], (w_shape))
            print('tamaño de los weigts',wsize)
            comp_weights = np.equal(w, weights)
            print('cantidad de elementos iguales', np.count_nonzero(comp_weights))
            print('cantidad de elementos diferentes', np.sum(comp_weights == 0))
            b=np.reshape(itm_fail[wsize:], (b_shape))
            print('shape bias_conv',b.shape)
            comp_bias = np.equal(b, biases)
            print('Bias cantidad de elementos iguales', np.count_nonzero(comp_bias))
            print('Bias cantidad de elementos diferentes', np.sum(comp_bias == 0))
            #print('compr_bias', comp_bias)
            weights_tmp.append(w)
            weights_tmp.append(b)
            Net2.layers[index].set_weights(weights_tmp)
            print('set weights')
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
loss,acc  = Net2.evaluate(test_dataset)                
print(Net2)
#Net_faulty = WeightsFaults(Net2,locs,error_mask,wfrac_size,wint_size)

        

Conv2D
tamaño d ela capa (None, 55, 55, 96)
shape wights_conv_antes (11, 11, 3, 96)
shape bias_antes_conv (96,)
inyectando errores en los pesos
numero de fallos insertados 134
tamaño de del array weights 34848
cantidad de pocisiones afectada capa 134
tamaño de la lista d eposiciones 134
tamaño de los weigts 34848
cantidad de elementos iguales 34715
cantidad de elementos diferentes 133
shape bias_conv (96,)
Bias cantidad de elementos iguales 96
Bias cantidad de elementos diferentes 0
set weights
BatchNormalization
Conv2D
tamaño d ela capa (None, 27, 27, 256)
shape wights_conv_antes (5, 5, 96, 256)
shape bias_antes_conv (256,)
inyectando errores en los pesos
numero de fallos insertados 6627
tamaño de del array weights 614400
cantidad de pocisiones afectada capa 6627
tamaño de la lista d eposiciones 6627
tamaño de los weigts 614400
cantidad de elementos iguales 607794
cantidad de elementos diferentes 6606
shape bias_conv (256,)
Bias cantidad de elementos iguales 256
Bias cantidad de eleme

# Testing code

### Comparacion entre modelos con y sin fallas


In [7]:
X = [x for x,y in test_dataset]
#salidas del modelo sin fallas para la primer imagen del dataset de prueba
outputs1 = get_all_outputs(Net1,X[0])
#salidas del modelo con fallas para la primer imagen del dataset de prueba
outputs2 = get_all_outputs(Net2,X[0])

In [8]:
#Comparando las salidas
for index in range(0,len(outputs1)):
    print('Capa',index,Net1.layers[index].__class__.__name__)
    print('maxima diferencia absoluta entre modelo con fallas y sin fallas: ',np.max(np.abs(outputs1[index]-outputs2[index])))

Capa 0 InputLayer
maxima diferencia absoluta entre modelo con fallas y sin fallas:  0.0
Capa 1 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  0.0
Capa 2 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  0.0
Capa 3 Conv2D
maxima diferencia absoluta entre modelo con fallas y sin fallas:  1023.98645
Capa 4 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  17.46289
Capa 5 ReLU
maxima diferencia absoluta entre modelo con fallas y sin fallas:  15.999512
Capa 6 BatchNormalization
maxima diferencia absoluta entre modelo con fallas y sin fallas:  506.44302
Capa 7 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  18.387207
Capa 8 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  18.387207
Capa 9 MaxPooling2D
maxima diferencia absoluta entre modelo con fallas y sin fallas:  18.250977
Capa 10 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  18.250977
Capa 11

In [18]:
def WeightsFaults(Net2,locs,error_mask,wfrac_size,wint_size):
    
    
    
    for index,layer in enumerate(Net2.layers):
        weights_tmp=[]
        print(Net2.layers[index].__class__.__name__)
        
        if(len(Net2.layers[index].weights) > 0):
            print(Net2.layers[index].__class__.__name__)
            if layer.__class__.__name__ in ['Conv2D','Conv1D','DepthwiseConv2D']:
                weights, biases = Net2.layers[index].get_weights()
                w_shape= weights.shape
                b_shape= biases
                wsize=weights.size
                weights_conv = np.reshape(weights, (-1))
                weig_conv_bias=np.concatenate((weights_conv, biases))
                itm= weig_conv_bias
                positionList,faultList,NumberOfFaults = GenerateFaultsList(shape=itm.shape,locs=locs,error_mask=error_mask)
                itm_fail= IntroduceFaultsInWeights(itm,positionList,faultList,wfrac_size,wint_size) 
                #d = itm_fail[:wsize]
                w=np.reshape(itm_fail[:wsize], (shape))
                b=np.reshape(itm_fail[wsize:], (shape))
                weights_tmp.append(w)
                weights_tmp.append(b)
                Net2.layers[index].set_weights(weights_tmp)
                
                
        #if  Net2.layers[index].__class__.__name__=='BatchNormalization':
        #        gamma,beta,moving_mean,moving_std = Net2.layers[index].get_weights()
        #        weights_0.append(gamma)
        #        weights_0.append(beta)
        #        weights_0.append(moving_mean)
        #        weights_0.append(moving_std)
        #        print(gamma.shape)
        #        print(beta.shape)
        #        print(moving_mean.shape)
        #        print(moving_std.shape)
        #        #gamma = np.reshape(gamma, (-1))
        #        #beta = np.reshape(beta, (-1))
        #        #moving_mean = np.reshape(moving_mean, (-1))
        #        #moving_std = np.reshape(moving_std, (-1))
        #        #weights_1=np.concatenate((gamma,beta,moving_mean,moving_std))
        #        #print('shape de weights_1:', weights_1.shape)  
        #     
        #    if  Net2.layers[index].__class__.__name__=='Dense':  
        #        weights_dense, biases_dense = Net2.layers[index].get_weights()
        #        weights_0.append(weights_dense)
        #        weights_0.append(biases_dense)
        #        print(weights_dense.shape)
        #        print(weights_dense.shape)
        #    
        #        #weights_dense = np.reshape(weights_dense, (-1))
        #        #biases_dense = np.reshape(biases_dense, (-1))
        #        #weights_2=np.concatenate((weights_dense, biases_dense))
        #        #print('weights_2', weights_2.shape)
        return Net2    
        

In [25]:
activation_aging = False

#cambiare la direccion del primer error y la posicion del fallo,
#seteandolo en una direccion baja (que afecte a la capa de normalizacion) y así poder observar la caida
#locs[0] = 0
#masks[0] = 'xx1xxxxxxxxxxxxx'
error_mask_address = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/error_mask_054')
locs_address = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/locs_054')


#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
#Introducir fallas en cada capa de pesos
weights = Net2.get_weights()
print(len(weights))
weights_0 = []
#print(weights_0)
for index,layer in enumerate(Net2.layers):
    print(Net2.layers[index].__class__.__name__)
    
    if(len(Net2.layers[index].weights) > 0):
        if layer.__class__.__name__ in ['Conv2D','Conv1D','DepthwiseConv2D']:
        #if Net2.layers[index].__class__.__name__=='Conv2D':
            weights, biases = Net2.layers[index].get_weights()
            weights_0.append(weights)
            weights_0.append(biases)
            print(weights.shape)
            print(weights.size)
            print(biases.shape)
            weights_conv = np.reshape(weights, (-1))
            #print('shape weights_conv',(weights_conv.shape))
            join_weights_conv_bias=np.concatenate((weights_conv, biases))
            #print('join perfecto', join_weights_conv_bias.shape)
            
        if  Net2.layers[index].__class__.__name__=='BatchNormalization':
            gamma,beta,moving_mean,moving_std = Net2.layers[index].get_weights()
            weights_0.append(gamma)
            weights_0.append(beta)
            weights_0.append(moving_mean)
            weights_0.append(moving_std)
            print(gamma.shape)
            print(beta.shape)
            print(moving_mean.shape)
            print(moving_std.shape)
            #gamma = np.reshape(gamma, (-1))
            #beta = np.reshape(beta, (-1))
            #moving_mean = np.reshape(moving_mean, (-1))
            #moving_std = np.reshape(moving_std, (-1))
            #weights_1=np.concatenate((gamma,beta,moving_mean,moving_std))
            #print('shape de weights_1:', weights_1.shape)  
         
        if  Net2.layers[index].__class__.__name__=='Dense':  
            weights_dense, biases_dense = Net2.layers[index].get_weights()
            weights_0.append(weights_dense)
            weights_0.append(biases_dense)
            print(weights_dense.shape)
            print(weights_dense.shape)
            #weights_dense = np.reshape(weights_dense, (-1))
            #biases_dense = np.reshape(biases_dense, (-1))
            #weights_2=np.concatenate((weights_dense, biases_dense))
            #print('weights_2', weights_2.shape)
## El tema está que a la hora d einyectar  los fallos como está diseñado se hace al modleo y de esta forma sería por capas
##entonces en este caso cuando se procese una capa deberiamos de mandar a inyectar los errores y luego agregarlo al modelo 
## ahi no entiendo como manejarlo
print(len(weights_0))
#positionList,faultList,NumberOfFaults = GenerateFaultsList(shape=itm.shape,locs=faulty_addresses,error_mask=masked_faults)
#
#    if NumberOfFaults > 0:
#        weights[index] = IntroduceFaultsInWeights(itm,positionList,faultList,wint_size,wint_size)
#qNet.set_weights(weights)
#loss = tf.keras.losses.CategoricalCrossentropy()
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
#Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
#loss,acc  = Net2.evaluate(test_dataset)
                    
            
        

36
InputLayer
Lambda
Lambda
Conv2D
(11, 11, 3, 96)
34848
(96,)
Lambda
ReLU
BatchNormalization
(96,)
(96,)
(96,)
(96,)
Lambda
Lambda
MaxPooling2D
Lambda
Conv2D
(5, 5, 96, 256)
614400
(256,)
Lambda
ReLU
BatchNormalization
(256,)
(256,)
(256,)
(256,)
Lambda
Lambda
MaxPooling2D
Lambda
Conv2D
(3, 3, 256, 384)
884736
(384,)
Lambda
ReLU
BatchNormalization
(384,)
(384,)
(384,)
(384,)
Lambda
Lambda
Conv2D
(1, 1, 384, 384)
147456
(384,)
Lambda
ReLU
BatchNormalization
(384,)
(384,)
(384,)
(384,)
Lambda
Lambda
Conv2D
(1, 1, 384, 256)
98304
(256,)
Lambda
ReLU
BatchNormalization
(256,)
(256,)
(256,)
(256,)
Lambda
Lambda
MaxPooling2D
Lambda
Flatten
Dense
(9216, 4096)
(9216, 4096)
Lambda
ReLU
Dropout
Lambda
Dense
(4096, 4096)
(4096, 4096)
Lambda
ReLU
Dropout
Lambda
Dense
(4096, 8)
(4096, 8)
Lambda
TFOpLambda
Lambda
36


### Crear la red con fallos en pesos y analizar los NAN

In [18]:
#v=54
#masked_faults = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/error_mask_0' + str(v))
#faulty_addresses = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/locs_0' + str(v))
#        #print('vol',vol)
#        #vol = vol - 1
#
# 
faulty_addresses = [16000]
error_mask = ['xx1xxxxxxxxxxxxx']
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=False, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
weights = Net2.get_weights()

for index,itm in enumerate(weights):
    positionList,faultList,NumberOfFaults = GenerateFaultsList(shape=itm.shape,locs=faulty_addresses,error_mask=masked_faults)
    #print('faultList:',faultList)
    if NumberOfFaults > 0:
     #print('NumberOfFaults:', NumberOfFaults)
        weights[index] = IntroduceFaultsInWeights(itm,positionList,faultList,wint_size,wint_size)
qNet.set_weights(weights)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
loss,acc  = Net2.evaluate(test_dataset)
        
        
      

inyectando errores en los pesos
inyectando errores en los pesos


InvalidArgumentError: indices[0] = [3, 464] does not index into param shape [4096,8], node name: GatherNd [Op:GatherNd]

### Crear la red con fallos en pesos

In [17]:
activation_aging = False

v=54
error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/error_mask_0' + str(v))
locs = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/locs_0' + str(v))
        #print('vol',vol)
        #vol = vol - 1

    
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=False, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
weights = Net2.get_weights()
for index,itm in enumerate(weights):
    print(index)
    print(itm.shape)
    
    

0
(11, 11, 3, 96)


ValueError: too many values to unpack (expected 2)

In [44]:
index=2
 #print('Capa',j,NetVecinos.layers[j].__class__.__name__)
weights = Net2.layers[index].get_weights()
capa=Net2.layers[index].__class__.__name__
print(capa)
print(weights)

Lambda
[]


¿Porque se da este error? analizandolo bien corresponde a un error en el modelado, cuando modele la introduccion de errores en los pesos lo hice de manera similar a las activaciones, modificando los valores capa a capa, sin embargo al obtener los pesos mediante .get_weights() la lista que se obtiene no esta ordenada capa a capa como en el caso de cuando obtenia las activaciones. 

(por suerte esto no afecto al trabajo que hice, ya que, estadisticamente este error solo cobra importancia cuando hay errores en bits significativos de direcciones muy bajas (400 o menor en el caso de AlexNet) que son una parte muy reducida con respecto al total de direcciones (885120 en AlexNet))

Para arreglar este error y que puedas continuar tu trabajo lo que tendrias que hacer es re-arreglar la lista obtenida por get_weights (presente en la variable weights del bloque de codigo anterior), si por ejemplola red tuviera las capas Conv->Batch Norm -> Dense entonces la weights estaria organizada así:
- weights[0] -> Filtros de la capa Convolucion (Dimension = 4)
- weights[1] -> Bias de la capa Convolucion    (Dimension = 1)
- weights[2] -> Parametros gamma de la capa Batch Normalization (Dimension = 1)
- weights[3] -> Parametros beta de la capa Batch Normalization  (Dimension = 1)
- weights[4] -> Parametros moving_mean de la capa Batch Normalization (Dimension = 1)
- weights[5] -> Parametros moving_std de la capa Batch Normalization  (Dimension = 1)
- weights[6] -> Pesos de la capa Dense (Dimension = 2)
- weights[7] -> Bias de la capa Dense (Dimension = 1)

Para que el modelo este correcto deberias modificar la variable weights y que quede así:
- weights[0] -> Filtros y Bias de la capa Convolucion (puedes vectorizarlos para que queden con Dimension = 1)
- weights[1] -> Parametros gamma,beta,moving_mean,moving_std de la capa Batch Normalization (puedes vectorizarlos para que queden con Dimension = 1 o unirlos en diferentes ejes y que queden con Dimension = 4)
- weights[2] -> Pesos y Bias de la capa Dense (puedes vectorizarlos queden con Dimension = 1)

Como veras en el ciclo for index,item del bloque anterior, con los pesos como estan ahora con una falla en 0, el error seria introducido multiples veces en cada capa, por ejemplo en el primer indice de filtros y en el primer indice de los bias; cuando deberia ser introducido solamente en uno de estos dos, dependiendo de cual asumimos que va primero en memoria, los filtros o los bias.

### Comparacion entre modelos con y sin fallas

In [84]:
X = [x for x,y in test_dataset]
#salidas del modelo sin fallas para la primer imagen del dataset de prueba
outputs1 = get_all_outputs(Net1,X[0])
#salidas del modelo con fallas para la primer imagen del dataset de prueba
outputs2 = get_all_outputs(Net2,X[0])

In [85]:
#Comparando las salidas
for index in range(0,len(outputs1)):
    print('Capa',index,Net1.layers[index].__class__.__name__)
    print('maxima diferencia absoluta entre modelo con fallas y sin fallas: ',np.max(np.abs(outputs1[index]-outputs2[index])))

Capa 0 InputLayer
maxima diferencia absoluta entre modelo con fallas y sin fallas:  0.0
Capa 1 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  0.0
Capa 2 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  0.0
Capa 3 Conv2D
maxima diferencia absoluta entre modelo con fallas y sin fallas:  3.7382812
Capa 4 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  3.7382812
Capa 5 ReLU
maxima diferencia absoluta entre modelo con fallas y sin fallas:  0.0
Capa 6 BatchNormalization
maxima diferencia absoluta entre modelo con fallas y sin fallas:  13.997774
Capa 7 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  13.997559
Capa 8 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  13.997559
Capa 9 MaxPooling2D
maxima diferencia absoluta entre modelo con fallas y sin fallas:  13.997559
Capa 10 Lambda
maxima diferencia absoluta entre modelo con fallas y sin fallas:  13.997559
Capa 11 Conv2